# 文本分类实例

## Step1 导入相关包

In [1]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification

## Step2 加载数据

In [2]:
import pandas as pd
data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [3]:
data.isnull().sum()

label     0
review    1
dtype: int64

In [4]:
data.dropna(inplace=True)
data.isnull().sum()

label     0
review    0
dtype: int64

## Step3 创建Dataset

In [5]:
from torch.utils.data import Dataset

class MyDataset(Dataset):

    def __init__(self) -> None:
        super().__init__()
        self.data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
        self.data = self.data.dropna()

    def __getitem__(self, index):
        return self.data.iloc[index]["review"], self.data.iloc[index]["label"]
    
    def __len__(self):
        return len(self.data)

In [6]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


## Step4 划分数据集

In [7]:
from torch.utils.data import random_split


trainset, validset = random_split(dataset, lengths=[0.9, 0.1])
len(trainset), len(validset)

(6989, 776)

In [8]:
for i in range(10):
    print(trainset[i])

('没酒店住，又住了一次。小姐骚扰从8点一次，变成两次，晚上9点多，和12点多。都睡觉了，气到想报警。前台告诉我，这个不是酒店责任。真搞笑，其他酒店都没有小姐骚扰，就华侨小姐上窜下跳，比老鼠还厉害。这次还有女同事同行，而隔壁房间的女同事没有接到任何骚扰电话。我该佩服小姐的智商还是该骂酒店卖客人呢？', 0)
('酒店离开市区挺远，很不方便。除了大堂气派，房间根本没有五星，充其量较差的四星，更莫名其妙的是房间竟然没有暖空调，下次决不考虑入住。', 0)
('不喜欢这个酒店，感觉同它的星级相差很远：1。没有门童，没人会来帮你提行李。2。前台态度冷漠，口气太生硬。3。我刚办了入住，把我的房号写错，害我提着行李上上下下的跑。4。自说自话把我同事的订单取消，携程出面交涉没有结果，很强硬。5。大堂太小，搞了些电脑让人上网。有些不象住店客人的人在那里上网，乌烟瘴气，像网吧。6。自以为聪明的推出了很多所谓的服务，但一般我们用不到。相反却完全不能给客人温馨的感觉。7。相对它的服务，这个价格太贵了！真不知道这样的酒店在奥运后怎样生存。。。', 0)
('应该不是我第一次评价这个酒店了，没办法再次补充如下：1、早餐真是超级垃圾，做得真的感觉和猪食差不多，不知道请的什么厨师？2、仍然是早餐的问题，在大堂旁的一个好象是叫‘玛格丽特的’西餐厅；1>光线特别不好，谁都希望早上起来有个明亮的环境用餐，但这里肯定不是；2>给食品加温的不是电热式，采用的是燃烧腊来加热/保温，弄得餐厅里面很浓的一股腊燃烧后的气味，让人感觉非常不好；3、真不知道这样的酒店怎么样就能够评上4星？哎。。。', 0)
('全家五一期间选择去富阳休闲两日,通过携程订了3个5月4日的标准房间,checkin的时候前台小姐告知下次来可不通过携程直接过来,感觉南国是很遵守游戏规则.你不要奢望当你下车的时候门童会主动来开车门和拿行李,即便是再多的行李,也不会等到行李车,服务生只是远远的看着,最让人不能接受的是该所谓的五星级酒店不开空调,晚上极为闷热,当询问前台时得到的答复竟然是"可能是因为你太怕热",因此强烈要求携程网在网上推荐时应在星号后边带上括号(常年不开空调,服务意识极差)', 0)
('交道便利，近民航售票处，房间大整洁，但早餐实在一般。', 1)
('房间里的设施没有广告中理想！价格也没有网上所说的那样分明清楚~~', 0)


## Step5 创建Dataloader

In [9]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def collate_func(batch):
    print(batch)
    texts, labels = [], []
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)
    return inputs

In [10]:
from torch.utils.data import DataLoader

trainloader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=collate_func)
validloader = DataLoader(validset, batch_size=64, shuffle=False, collate_fn=collate_func)

In [11]:
next(enumerate(validloader))[1]

{'input_ids': tensor([[ 101,  857, 4638,  ...,  679, 7231,  102],
        [ 101, 6006, 4197,  ..., 2168,  511,  102],
        [ 101, 1126,  725,  ...,    0,    0,    0],
        ...,
        [ 101, 6983, 2421,  ...,    0,    0,    0],
        [ 101, 6163,  934,  ...,    0,    0,    0],
        [ 101,  127, 3517,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 1, 1

## Step6 创建模型及优化器

In [12]:
from torch.optim import Adam

model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")

if torch.cuda.is_available():
    model = model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
optimizer = Adam(model.parameters(), lr=2e-5)

## Step7 训练与验证

In [14]:
def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in validloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return acc_num / len(validset)

def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_step == 0:
                print(f"ep: {ep}, global_step: {global_step}, loss: {output.loss.item()}")
            global_step += 1
        acc = evaluate()
        print(f"ep: {ep}, acc: {acc}")

## Step8 模型训练

In [15]:
train()

ep: 0, global_step: 0, loss: 0.68853759765625
ep: 0, global_step: 100, loss: 0.4265013039112091
ep: 0, global_step: 200, loss: 0.19690829515457153
ep: 0, acc: 0.8917525410652161
ep: 1, global_step: 300, loss: 0.21174821257591248
ep: 1, global_step: 400, loss: 0.3158130347728729
ep: 1, acc: 0.9046391248703003
ep: 2, global_step: 500, loss: 0.1521395891904831
ep: 2, global_step: 600, loss: 0.24524062871932983
ep: 2, acc: 0.8981958627700806


## Step9 模型预测

In [16]:
sen = "我觉得这家酒店不错，饭很好吃！"
id2_label = {0: "差评！", 1: "好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"输入：{sen}\n模型预测结果:{id2_label.get(pred.item())}")

输入：我觉得这家酒店不错，饭很好吃！
模型预测结果:好评！


In [17]:
from transformers import pipeline

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [18]:
pipe(sen)

[{'label': '好评！', 'score': 0.9968716502189636}]